<a href="https://colab.research.google.com/github/ziad11222/Grocery_Project/blob/main/Graduation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from keras.utils import to_categorical


In [ ]:
from tensorflow.keras.layers import Conv2D, UpSampling2D, Conv2DTranspose

In [ ]:
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras import Model



In [ ]:
import sys
sys.path.append("/content/thou/")
import logging
import os
from random import randint
from keras import backend as K
import random
from PIL import Image
from keras.models import load_model
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
import pickle

In [ ]:
!unzip '/content/drive/My Drive/images.zip' -d '/content/'

unzip:  cannot find or open /content/drive/My Drive/images.zip, /content/drive/My Drive/images.zip.zip or /content/drive/My Drive/images.zip.ZIP.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unrar x '/content/content/ThoughtViz.rar'

In [ ]:
!zip -r thou2.zip /content/thou


  adding: content/thou/ (stored 0%)
  adding: content/thou/training/ (stored 0%)
  adding: content/thou/training/baseline_acgan_with_eeg.py (deflated 68%)
  adding: content/thou/training/baseline_deligan_image.py (deflated 70%)
  adding: content/thou/training/__init__.py (stored 0%)
  adding: content/thou/training/__pycache__/ (stored 0%)
  adding: content/thou/training/__pycache__/__init__.cpython-310.pyc (deflated 27%)
  adding: content/thou/training/eeg_classification.py (deflated 62%)
  adding: content/thou/training/thoughtviz_with_eeg.py (deflated 69%)
  adding: content/thou/training/thoughtviz_with_label.py (deflated 68%)
  adding: content/thou/training/thoughtviz_image_with_eeg.py (deflated 70%)
  adding: content/thou/training/baseline_acgan_image.py (deflated 70%)
  adding: content/thou/training/models/ (stored 0%)
  adding: content/thou/training/models/__init__.py (stored 0%)
  adding: content/thou/training/models/thoughtviz.py (deflated 81%)
  adding: content/thou/training/mo

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the pre-trained model from an h5 file
model = load_model("run_final.h5")
pkl_path = '/content/data/eeg/char/data.pkl'


OSError: No file or directory found at run_final.h5

In [ ]:
eeg_data = pickle.load(open(pkl_path, "rb"), encoding='bytes')


In [ ]:
model.summary()


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model

# Load your pre-trained model from an h5 file (replace with your actual model path)
model = load_model("run_final.h5")

# Plot the model architecture and save it to a file (optional)
plot_model(model, to_file="model_diagram.png", show_shapes=True, show_layer_names=True)

# Display the model diagram (requires Graphviz to be installed)
# If you haven't installed Graphviz, you can do so using: !pip install graphviz
plot_model(model, show_shapes=True, show_layer_names=True)


In [ ]:
x_train, y_train, x_test, y_test = inutil.load_char_data('/content/images/Char-Font/', resize_shape=(28, 28))
print("Loaded Characters Dataset.", )

In [ ]:
Ayhaja = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_val, y_val))


In [ ]:
pip install --upgrade keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.2.1 which is incompatible.


In [ ]:
import logging
import os
from random import randint
from keras import backend as K
import random
from PIL import Image
from keras.models import load_model
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
import pickle
import thou.utils.data_input_util as inutil
from   thou.training.models.thoughtviz import *
from   thou.utils.image_utils import *
from   thou.utils.eval_utils import *



>> Downloading inception-2015-12-05.tgz 100.0%
Succesfully downloaded inception-2015-12-05.tgz 88931400 bytes.


In [39]:


def train_gan(input_noise_dim, batch_size, epochs, model_save_dir, output_dir, classifier_model_file, **kwargs):
    # Function body
    # You can access splits_save_dir with kwargs.get('splits_save_dir', default_value)
    # Add your existing function logic here

    K.set_learning_phase(False)
    # folders containing images used for training
    imagenet_folder = "/content/ThoughtViz/training/images/ImageNet-Filtered"
    num_classes = 10

    feature_encoding_dim = 100

    # load data and compile discriminator, generator models depending on the dataaset
    x_train, y_train, x_test, y_test = inutil.load_image_data(imagenet_folder, patch_size=(64, 64))
    print("Loaded Images Dataset.", )

    g_adam_lr = 0.00003
    g_adam_beta_1 = 0.5

    d_adam_lr = 0.00005
    d_adam_beta_1 = 0.5

    c = load_model(classifier_model_file)

    d = discriminator_model_rgb((64, 64), c)
    d_optim = Adam(lr=d_adam_lr, beta_1=d_adam_beta_1)
    d.compile(loss=['binary_crossentropy','categorical_crossentropy'], optimizer=d_optim)
    d.trainable = True

    g = generator_model_rgb(input_noise_dim, feature_encoding_dim)
    g_optim = Adam(lr=g_adam_lr, beta_1=g_adam_beta_1)
    g.compile(loss='categorical_crossentropy', optimizer=g_optim)

    d_on_g = generator_containing_discriminator(input_noise_dim, feature_encoding_dim, g, d)
    d_on_g.compile(loss=['binary_crossentropy','categorical_crossentropy'], optimizer=g_optim)

    g.summary()
    d.summary()

    eeg_data = pickle.load(open(os.path.join(data_dir, 'data.pkl'), "rb"))
    classifier = load_model(saved_classifier_model_file)
    classifier.summary()
    x_test = eeg_data[b'x_test']
    y_test = eeg_data[b'y_test']
    y_test = np.array([np.argmax(y) for y in y_test])
    layer_index = 9

    # keras way of getting the output from an intermediate layer
    get_nth_layer_output = K.function([classifier.layers[0].input], [classifier.layers[layer_index].output])

    layer_output = get_nth_layer_output([x_test])[0]

    for epoch in range(epochs):
        print("Epoch is ", epoch)

        print("Number of batches", int(x_train.shape[0]/batch_size))

        for index in range(int(x_train.shape[0]/batch_size)):
            # generate noise from a normal distribution
            noise = np.random.uniform(-1, 1, (batch_size, input_noise_dim))

            random_labels = np.random.randint(0, 10, batch_size)

            one_hot_vectors = [to_categorical(label, 10) for label in random_labels]

            eeg_feature_vectors = np.array([layer_output[random.choice(np.where(y_test == random_label)[0])] for random_label in random_labels])

            # get real images and corresponding labels
            real_images = x_train[index * batch_size:(index + 1) * batch_size]
            real_labels = y_train[index * batch_size:(index + 1) * batch_size]

            # generate fake images using the generator
            generated_images = g.predict([noise, eeg_feature_vectors], verbose=0)

            # discriminator loss of real images
            d_loss_real = d.train_on_batch(real_images, [np.array([1] * batch_size), np.array(real_labels)])
            # discriminator loss of fake images
            d_loss_fake = d.train_on_batch(generated_images, [np.array([0] * batch_size), np.array(one_hot_vectors).reshape(batch_size, num_classes)])
            d_loss = (d_loss_fake[0] + d_loss_real[0]) * 0.5

            d.trainable = False
            # generator loss
            g_loss = d_on_g.train_on_batch([noise, eeg_feature_vectors], [np.array([1] * batch_size), np.array(one_hot_vectors).reshape(batch_size, num_classes)])
            d.trainable = True

        # save generated images at intermediate stages of training
        if epoch % 100 == 0:
            image = combine_rgb_images(generated_images)
            image = image * 255.0
            img_save_path = os.path.join(output_dir, str(epoch) + "_g" + ".png")
            Image.fromarray(image.astype(np.uint8)).save(img_save_path)

        if epoch % 100 == 0:
            test_image_count = 50000
            test_noise = np.random.uniform(-1, 1, (test_image_count, input_noise_dim))
            test_labels = np.random.randint(0, 10, test_image_count)

            eeg_feature_vectors_test = np.array([layer_output[random.choice(np.where(y_test == test_label)[0])] for test_label in test_labels])
            test_images = g.predict([test_noise, eeg_feature_vectors_test], verbose=0)
            test_images = test_images * 255.0
            inception_score = get_inception_score([test_image for test_image in test_images], splits=10)

        print("Epoch %d d_loss : %f" % (epoch, d_loss))
        print("Epoch %d g_loss : %f" % (epoch, g_loss[0]))
        print("Epoch %d inception_score : %f" % (epoch, inception_score[0]))

        if epoch % 50 == 0:
            # save generator and discriminator models along with the weights
            g.save(os.path.join(model_save_dir, 'generator_' + str(epoch)), overwrite=True, include_optimizer=True)
            d.save(os.path.join(model_save_dir, 'discriminator_' + str(epoch)), overwrite=True, include_optimizer=True)


def train():
    dataset = 'Image'
    batch_size = 100
    run_id = 1
    epochs = 10000
    eeg_data_dir = '/content/ThoughtViz/data/eeg/image/data.pkl'
    eeg_classifier_model_file = '/content/ThoughtViz/models/eeg_models/image/run_final.h5'
    output_dir = '/content/content'
    classifier_model_file = '/content/thou/training/models/classification.py'
    model_save_dir = os.path.join('./saved_models/thoughtviz_image_with_eeg/', dataset, 'run_' + str(run_id))
    train_gan(input_noise_dim=100, batch_size=batch_size, epochs=epochs, splits_save_dir=eeg_data_dir, saved_classifier_model_file=eeg_classifier_model_file, model_save_dir=model_save_dir, output_dir=output_dir, classifier_model_file=classifier_model_file)

    if not os.path.exists(model_save_dir):
        os.makedirs(model_save_dir)

    output_dir = os.path.join('./outputs/thoughtviz_image_with_eeg/', dataset, 'run_' + str(run_id))
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    classifier_model_file = os.path.join('./trained_classifier_models', 'classifier_' + dataset.lower() + '.h5')

    eeg_data_dir = os.path.join('../data/eeg/', dataset.lower())
    eeg_classifier_model_file = os.path.join('../models/eeg_models', dataset.lower(), 'run_final.h5')

    train_gan(input_noise_dim=100, batch_size=batch_size, epochs=epochs, splits_save_dir=eeg_data_dir, saved_classifier_model_file=eeg_classifier_model_file, model_save_dir=model_save_dir, output_dir=output_dir, classifier_model_file=classifier_model_file)


if __name__ == '__main__':
    train()


1796 1796
Loaded Images Dataset.


OSError: Unable to open file (file signature not found)

In [26]:
import pickle
import random
import keras.backend as K
from PIL import Image
from keras.models import load_model
from keras.utils import to_categorical

from layers.mog_layer import *
from utils.image_utils import *


class Tests():

    def test_deligan_baseline(self, generator_model):
        K.set_learning_phase(False)

        input_noise_dim = 100
        batch_size = 50

        noise = np.random.uniform(-1, 1, (batch_size, input_noise_dim))

        random_labels = np.random.randint(0, 10, batch_size)

        conditioned_noise = []
        for i in range(batch_size):
            conditioned_noise.append(np.append(noise[i], to_categorical(random_labels[i], 10)))
        conditioned_noise = np.array(conditioned_noise)

        g = load_model(generator_model, custom_objects={'MoGLayer': MoGLayer})

        # generate images using the generator
        generated_images = g.predict(conditioned_noise, verbose=0)

        image = combine_rgb_images(generated_images)
        image = image * 127.5 + 127.5
        img = Image.fromarray(image.astype(np.uint8))
        img.show()

    def test_deligan_final(self, generator_model, classifier_model, eeg_pkl_file):
        K.set_learning_phase(False)

        # load EEG data
        eeg_data = pickle.load(open(eeg_pkl_file, "rb"), encoding='bytes')
        classifier = load_model(classifier_model)

        x_test = eeg_data[b'x_test']
        y_test = eeg_data[b'y_test']
        y_test = [np.argmax(y) for y in y_test]
        layer_index = 9

        # keras way of getting the output from an intermediate layer
        get_nth_layer_output = K.function([classifier.layers[0].input], [classifier.layers[layer_index].output])

        layer_output = get_nth_layer_output([x_test])[0]

        input_noise_dim = 100
        batch_size = 50

        noise = np.random.uniform(-1, 1, (batch_size, input_noise_dim))

        random_labels = np.random.randint(0, 10, batch_size)

        eeg_feature_vectors = [layer_output[random.choice(np.where(y_test == random_label)[0])] for random_label in random_labels]

        noises, conditionings = [], []
        for i in range(batch_size):
            noises.append(noise[i])
            conditionings.append(eeg_feature_vectors[i])

        g = load_model(generator_model, custom_objects={'MoGLayer': MoGLayer})

        # generate images using the generator
        generated_images = g.predict([np.array(noises), np.array(conditionings)], verbose=0)

        image = combine_rgb_images(generated_images)
        image = image * 127.5 + 127.5
        img = Image.fromarray(image.astype(np.uint8))
        img.show()


if __name__ == '__main__':
    tests = Tests()
    #tests.test_deligan_baseline('../models/gan_models/baseline/deligan/image/generator.model')
    tests.test_deligan_final('../models/gan_models/final/image/generator.model',
                       '/content/ThoughtViz/models/eeg_models/image/run_final.h5',
                       '/content/ThoughtViz/data/eeg/image/data.pkl')


UnimplementedError: Exception encountered when calling layer 'conv2d_3' (type Conv2D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:CPU:0}} The Conv2D op currently only supports the NHWC tensor format on the CPU. The op was given the format: NCHW [Op:Conv2D] name: 

Call arguments received by layer 'conv2d_3' (type Conv2D):
  • inputs=tf.Tensor(shape=(5706, 1, 9, 25), dtype=float32)